<a href="https://colab.research.google.com/github/yorkjong/vistock/blob/feature%2Fmansfield/notebooks/rsm_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with RSM (Mansfield Relative Strength) Rating

### Install and Config

#### Install Required Packages

In [8]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/mansfield"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/mansfield) to /tmp/pip-req-build-hfrvfx5r
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-hfrvfx5r
  Running command git checkout -b feature/mansfield --track origin/feature/mansfield
  Switched to a new branch 'feature/mansfield'
  Branch 'feature/mansfield' set up to track remote branch 'feature/mansfield' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit cec7348a02ac086991e0a41b2b3b974f61fb4003
  Preparing metadata (setup.py) ... done


#### Setup and Configuration

In [9]:
# @title Enable DataFrame Formatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [10]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [11]:
# @title GitHub
import base64
import requests
import pandas as pd
from io import StringIO


class GitHub:
    def __init__(self, repo_owner, repo_name, token, dir='', branch='main'):
        base = 'https://api.github.com/repos'
        dir = dir.strip('/')
        if dir:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch
        self.token = token

    def _request(self, method, url, headers=None, params=None, json=None):
        response = requests.request(method, url, headers=headers,
                                    params=params, json=json)
        if response.status_code in [200, 201]:
            return response.json()
        elif response.status_code == 404:
            return None
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def file_exists(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            return True
        elif response.status_code == 404:
            return False
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def list_filenames(self, dir_path=''):
        url = f'{self.base_url}/{dir_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return []

    def download_file(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }

        file_info = self._request('GET', url, headers=headers,
                                  params={'ref': self.branch})
        if file_info:
            response = requests.get(file_info['download_url'])
            if response.status_code == 200:
                return StringIO(response.text)
            else:
                print(f"Failed to download file: "
                      f"{response.status_code} - {response.text}")
        else:
            print(f"File '{file_path}' does not exist. Cannot download.")
        return None

    def download_csv(self, file_path):
        file_content = self.download_file(file_path)
        if file_content:
            return pd.read_csv(file_content)
        else:
            return pd.DataFrame()

    def upload_file(self, file_path, content):
        url = f'{self.base_url}/{file_path}'

        # Encode the content to base64
        encoded_content = base64.b64encode(content.encode()).decode()
        payload = {
            'message': 'Uploading file',
            'content': encoded_content,
            'branch': self.branch
        }

        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Check if the file already exists to get the current sha
        file_info = self._request('GET', url, headers=headers,
                                  params={'ref': self.branch})
        # If the file exists, get the current SHA
        if file_info:
            payload['sha'] = file_info.get('sha')

        # PUT request to create or update the file
        self._request('PUT', url, headers=headers, json=payload)

    def upload_df_as_csv(self, file_path, df):
        """Upload a DataFrame to a CSV file."""
        if not file_path.endswith('.csv'):
            file_path += '.csv'
        csv_content = df.to_csv(index=False)
        self.upload_file(file_path, csv_content)

    def remove_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Skipping deletion.")
            return

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Fetch the file info to get the SHA needed for deletion
        file_info = self._request('GET', url, headers=headers)
        if file_info:
            payload = {
                'message': 'Deleting file',
                'sha': file_info['sha'],
                'branch': self.branch
            }
            self._request('DELETE', url, headers=headers, json=payload)

#-------------------------------------------------------------------------------

from google.colab import userdata

github = GitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    token=userdata.get('GithubToken.stock-reports'),
    dir='weinstein',
)

In [12]:
# @title DataFrame Operations

def is_taiwan_stock_df(df):
    ticker = df['Ticker'].iloc[0].replace('.TWO', '').replace('.TW', '')
    return ticker.isdigit()

def add_name_column(df):
    column_names = df.columns.tolist()
    if 'Name' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    if df.empty:
        return df
    if not is_taiwan_stock_df(df):
        return df
    df['Name'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Name')
    df = df[column_names]   # create a new DataFrame
    df['Name'] = df['Ticker'].apply(tw.stock_name)
    return df


def remove_ticker_suffix(df, ticker_column):
    """
    Remove the '.TW' or '.TWO' suffix from ticker codes in a DataFrame.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    """
    # Apply string replacement for each ticker in the specified column
    df[ticker_column] = df[ticker_column].str.replace('.TWO', '', regex=False)
    df[ticker_column] = df[ticker_column].str.replace('.TW', '', regex=False)


def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]


In [13]:
# @title Rank Function

import os
from datetime import datetime

from vistock import rsm
from vistock import tw
from vistock.stock_indices import get_tickers


def remove_failed_tickers(tickers):
    delisted = ['BRK.B', 'LEN.B', 'BF.B', 'UHAL.B', 'BF.A', 'CWEN.A', 'HEI.A']
    invalid = ['GEV', 'SOLV', 'VLTO', 'SW', 'ARM', 'CART', 'AS', 'BIRK', 'VSTS','LOAR', 'ALAB','GRAL', 'SEG']
    invalid += ['00945B.TW', '6928.TW', '6914.TW', '6771.TW', '00944.TW', '8162.TW', '1563.TW', '00946.TW', '00941.TW', '6423.TW', '00940.TW', '00939.TW', '4949.TW', '00943.TW', '8487.TW', '6794.TW', '6949.TW', '4771.TW']
    invalid += ['00936.TW', '6805.TW', '2254.TW', '6658.TW', '00935.TW', '6592B.TW', '6526.TW', '6906.TW', '4736.TW', '00636K.TW', '6968.TWO', '4442.TWO', '6534.TW', '6901.TW', '00934.TW', '00657K.TW', '6472.TW', '2258.TW', '6916.TW', '2762.TW', '6933.TW']
    invalid += ['02001R.TW', '020031.TW', '020039.TW', '020016.TW', '02001L.TW', '020019.TW', '020028.TW', '020020.TW', '02001S.TW', '020018.TW', '020038.TW', '020034.TW', '020011.TW', '020030.TW', '020012.TW', '020036.TW', '020029.TW', '020000.TW', '020015.TW', '020037.TW']
    invalid += ['6890.TW', '00951.TW', '3150.TW', '6957.TW', '00947.TW', '00949.TW']
    invalid += ['6838.TW', '00953B.TW', '00956.TW', '00954.TW']
    return list(set(tickers) - set(delisted) - set(invalid))

def rank(code, period='2y', ma="EMA", tickers_getter=get_tickers,
         ticker_ref='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)
    tickers = remove_failed_tickers(tickers)

    rank_stock = rsm.ranking(tickers, period=period,
                             ticker_ref=ticker_ref, ma=ma)
    if rank_stock.empty:
        print("Not enough data to generate rankings.")
        return

    rank_stock = add_name_column(rank_stock)

    # Remove the '.TW' or '.TWO' suffix
    remove_ticker_suffix(rank_stock, 'Ticker')

    # Save to CSV
    print("\n\n***")
    os.makedirs(out_dir, exist_ok=True)
    today = datetime.now().strftime('%Y%m%d')
    filename = f'{code}_stocks_{period}_{ma}_{today}.csv'
    github.upload_df_as_csv(filename, rank_stock)
    rank_stock.to_csv(os.path.join(out_dir, filename), index=False)
    print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock

### RS Rating and Ranking

In [14]:
source = "PHLX Semiconductor" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", 'Russell 1000', "PHLX Semiconductor", "All Indices"]
ma = "SMA" # @param ["EMA","SMA"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'Russell 1000': 'RUI',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'SPX+DJIA+NDX+RUI+SOX',
}

df = rank(code_from_name[source], ma=ma)
data_table.DataTable(df, include_index=False, num_rows_per_page=20)

[*********************100%***********************]  31 of 31 completed


[**********************100%**********************]  30 of 30 info downloaded


***
Your "SOX_stocks_2y_SMA_20240828.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Rank,1 Month Ago,3 Months Ago,6 Months Ago,Price
28,NVDA,Technology,Semiconductors,45.98,42.69,62.98,61.71,1,1,1,1,128.30
11,COHR,Technology,Scientific & Technical Instruments,34.98,27.93,15.87,36.31,2,2,9,4,79.48
8,MPWR,Technology,Semiconductors,26.85,17.57,15.51,22.01,3,4,10,9,945.46
13,TSM,Technology,Semiconductors,17.70,19.34,28.77,17.86,4,3,4,12,170.55
1,AVGO,Technology,Semiconductors,15.91,16.46,19.60,36.51,5,5,8,3,161.39
17,KLAC,Technology,Semiconductor Equipment & Materials,11.28,13.76,22.77,26.55,6,6,5,7,812.79
15,TXN,Technology,Semiconductors,7.12,5.44,5.07,-11.63,7,8,17,24,210.69
12,TER,Technology,Semiconductor Equipment & Materials,6.16,2.44,21.79,-9.66,8,14,6,23,134.98
21,ADI,Technology,Semiconductors,2.96,3.44,9.23,-8.33,9,13,15,21,231.84
27,NOVT,Technology,Scientific & Technical Instruments,0.45,4.07,-9.12,-4.05,10,10,20,20,182.01


### RS Rating and Ranking for Taiwan Stocks

In [ ]:
from vistock import tw

source = "上市+上櫃" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]
ma = "SMA" # @param ["EMA","SMA"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

df_tw = rank(code_from_name[source], ma=ma,
             tickers_getter=tw.get_tickers, ticker_ref='^TWII')
data_table.DataTable(df_tw, include_index=False, num_rows_per_page=20)

[*********************100%***********************]  2007 of 2007 completed


..................................................


***
Your "TWSE+TPEX_stocks_2y_SMA_20240827.csv" is in the "out" folder.
***



,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Rank,1 Month Ago,3 Months Ago,6 Months Ago
404,6144,得利影,98.90,Communication Services,Entertainment,235.09,294.06,32.92,-6.31,1,1,89,927
743,3230,錦明,43.15,Technology,Electronic Components,179.48,65.97,5.86,-16.88,2,45,332,1626
1154,1799,易威,219.50,Healthcare,Drug Manufacturers—Specialty & Generic,167.61,128.32,122.20,72.36,3,8,3,33
1411,6640,均華,1005.00,Technology,Semiconductors,162.56,164.43,90.39,130.47,4,5,10,10
704,8374,羅昇,137.50,Industrials,Industrial Distribution,150.52,169.24,13.48,-12.89,5,4,228,1394
...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,6291,沛亨,123.50,Technology,Semiconductors,-43.05,-44.54,-37.29,-12.58,2002,2004,1975,1364
1405,2740,天蔥,24.00,Consumer Cyclical,Restaurants,-43.16,-35.84,-26.76,-24.13,2003,1980,1805,1886
1519,5210,寶碩,23.30,Technology,Software—Application,-44.95,-39.09,-5.66,42.30,2004,1995,652,86
1173,3308,聯德,24.65,Industrials,Electrical Equipment & Parts,-45.23,-47.19,-54.08,-18.13,2005,2005,1996,1687


### Remove files in GitHub Repository

In [ ]:
# @title CSV Deleter
import re
import ipywidgets as widgets

# Example filenames
with requests_cache.disabled():
    all_filenames = github.list_filenames()

# Function to extract unique dates from filenames
def extract_dates(filenames):
    date_pattern = r'\d{8}'
    dates = set()
    for fn in filenames:
        match = re.search(date_pattern, fn)
        if match:
            dates.add(match.group(0))
    return sorted(dates, reverse=True)  # Sort dates from newest to oldest

# Function to remove a file (replace with your actual implementation)
def remove_file(filename):
    print(f"Removing file: {filename}")
    with requests_cache.disabled():
        github.remove_file(filename)
    all_filenames.remove(filename)

#-------------------------------------------------------------------------------

# Update file selector options based on selected date
def update_file_selector(change):
    def selector_width(filenames):
        max_filename_length = max(len(fn) for fn in filenames)
        return f'{max_filename_length * 10}px'  # 10px width per character

    selected_date = change['new']
    lst_fns = [fn for fn in all_filenames if selected_date in fn]
    file_selector_widget.options = lst_fns
    file_selector_widget.rows = len(file_selector_widget.options)
    file_selector_widget.layout=widgets.Layout(width=selector_width(lst_fns))

# Function to delete selected files
def delete_files(button):
    selected_files = file_selector_widget.value
    for file in selected_files:
        remove_file(file)
    update_widgets()    # Update widgets after deletion

# Update widgets to reflect current state
def update_widgets():
    # Refresh the date selector
    dates = extract_dates(all_filenames)
    selected_date = date_selector_widget.value
    if dates and selected_date not in dates:
        i = date_selector_widget.options.index(selected_date)
        if i > len(dates) - 1:
            selected_date = dates[-1]
        else:
            selected_date = dates[i]
    date_selector_widget.options = dates
    date_selector_widget.value = selected_date
    if not dates:
        file_selector_widget.options = []
        return
    update_file_selector({'new': date_selector_widget.value})

#-------------------------------------------------------------------------------

# Create a widget for selecting dates
def create_date_selector(dates):
    return widgets.Dropdown(
        options=dates,
        value = dates[0] if dates else None,
        description='Date:',
        disabled=False
    )

# Create a widget for selecting files
def create_file_selector(filenames):

    return widgets.SelectMultiple(
        options=[],
        value=[],
        description='Files',
        disabled=False,
    )

# Create widgets
dates = extract_dates(all_filenames)
date_selector_widget = create_date_selector(dates)
file_selector_widget = create_file_selector(all_filenames)
delete_button = widgets.Button(description="Delete Selected Files")
delete_button.on_click(delete_files)

# Initialize the file selector with the latest date
update_widgets()

# Set up the observer to update file selector when date is changed
date_selector_widget.observe(update_file_selector, names='value')

# Display widgets
display(date_selector_widget, file_selector_widget, delete_button)

Dropdown(description='Date:', options=('20240827',), value='20240827')

SelectMultiple(description='Files', layout=Layout(width='470px'), options=('SPX+DJIA+NDX+RUI+SOX_stocks_2y_SMA…

Button(description='Delete Selected Files', style=ButtonStyle())